# PART 4: POS Tagging on a non-English language

_by Nicolas Bessone

In [1]:
# Imports
%load_ext autoreload
%autoreload 2

import re
import os

import pandas as pd

import nltk
import spacy

from textblob import TextBlob
from textblob.taggers import PatternTagger

import skweak

from scripts.skweak_ner_eval import evaluate
from scripts.utils import load_data_split, get_frequent_words, tag_all, penntreebank2universal, compute_recall, compute_num_conflicts

pd.set_option('display.max_rows', 500)

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
!python -m textblob.download_corpora

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
     ---------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.8 MB 2.8 MB/s eta 0:00:05
     -- ------------------------------------- 0.9/12.8 MB 3.7 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/12.8 MB 4.3 MB/s eta 0:00:03
     --- ------------------------------------ 1.3/12.8 MB 4.5 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 4.2 MB/s eta 0:00:03
     ----- ---------------------------------- 1.9/12.8 MB 4.4 MB/s eta 0:00:03
     ------- -------------------------------- 2.3/12.8 MB 5.3 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.8 MB 5.3 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.8 MB

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\nhbes\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!

## POS tags

For this tutorial, we will use the following subset of the [universal POS tags](https://universaldependencies.org/u/pos/index.html):
1. **DET**: determiner, which is a word that modifies nouns or noun phrases and expresses the reference of the noun phrase in context.
2. **NUM**: numeral. It is a word that expresses a number and a relation to the number, such as quantity, sequence, frequency or fraction.
3. **PROPN**: proper noun is a noun that is the name of a specific individual, place, or object.
4. **ADJ**: adjective, which is a word that typically modifies nouns and specifies their properties or attributes.
5. **NOUN**: noun, which is a part of speech typically denoting a person, place, thing, animal or idea.

In [3]:
all_labels = ["DET", "NUM", "PROPN", "ADJ", "NOUN"]

## Load data

We will use the [English corpus](https://universaldependencies.org/treebanks/en_ewt/index.html) from Universal Dependencies, a framework that contains consistent grammatical annotations across many different languages.
The texts in the corpus come from five types of web media: weblogs, newsgroups, emails, reviews, and Yahoo! answers and consist of 254,825 words and 16,621 sentences.

Skweak operates on spaCy ``doc`` objects, so the dataset is loaded in this format.

In [4]:
# Load training data
train_docs = load_data_split("train", all_labels)

In [5]:
for doc in train_docs[:3]:
    skweak.utils.display_entities(doc)

## 3.1 Labeling functions

In the first step, we find the 200 most frequent words in our training corpus and use a lexicon to label these words. In the second step, we mannually annotate the 50 most frequent words.
Finally, for each POS tag we will create the following labeling functions: 

*   DET --> Lexicon with determiners.
*   NUM --> If the token is a number or a word indicating a number from 1 to 10.
*   PROPN --> A word that is capitalized.
*   ADJ --> List of prefixes and suffixes. Syntactic rules that check: 1. if the previous word is a form of "be" and 2. if the previous word is a determiner or numeral.
*   NOUN --> List of prefixes and suffixes. Syntactic rule checking if the previous word is a determiner, numeral or adjective.

#### Lexicon LF

In [6]:
# Get the 200 most frequent words in the training set
frequent_words = get_frequent_words(train_docs, 200)
print(frequent_words[:5])

['de', 'la', 'el', 'en', 'que']


In [7]:
# Load the lexicon
with open("noun_vb_adj_spanish.txt") as f:
    lines = f.readlines()

# Create a dictionary with the words and their pos tags
lexicon = {}
for l in lines:
    values = l.replace("\n", "").split("\t")
    lexicon[values[0]] = values[1]

In [8]:
print("There are", len(lexicon), "words in the lexicon.")
print(list(lexicon.items())[:5])

There are 594 words in the lexicon.
[('ser', 'VERB'), ('haber', 'VERB'), ('tener', 'VERB'), ('poder', 'VERB'), ('estar', 'VERB')]


In [9]:
# How many of the frequent words we found exist in the lexicon
len((list(set(frequent_words) & set(list(lexicon.keys())))))

47

In [10]:
# Lexicon LF
def frequent_word_detector(doc):
    for token in doc:
        # If the frequent word exists in the lexicon use its assigned pos tag
        if token.text.lower() in frequent_words and token.text.lower() in list(lexicon.keys()):
            yield token.i, token.i + 1, lexicon[token.text.lower()]


lexicon_lf = skweak.heuristics.FunctionAnnotator("frequent_words", frequent_word_detector)


#### Manual annotation LF

In [11]:
# Manual annotation
top50_words = get_frequent_words(train_docs, 50)
print(top50_words)

['de', 'la', 'el', 'en', 'que', 'los', 'del', 'se', 'un', 'por', 'con', 'una', 'las', 'es', 'su', 'para', 'al', 'como', 'fue', 'más', 'lo', 'sus', 'este', 'ha', 'pero', 'también', 'esta', 'entre', 'son', 'dos', 'le', 'muy', 'años', 'desde', 'año', 'hasta', 'era', 'sobre', 'durante', 'ser', 'eran', 'sin', 'donde', 'ya', 'está', 'tiene', 'parte', 'población', 'km', 'ciudad']


In [12]:
manual_tags = {
    'de': 'DET',
    'la': 'DET',
    'el': 'DET',
    'en': 'DET',
    'que': 'DET',
    'los': 'DET',
    'del': 'DET',
    'se': 'DET',
    'un': 'NUM',
    'por': 'DET',
    'con': 'DET',
    'una': 'NUM',
    'las': 'DET',
    'es': 'DET',
    'su': 'DET',
    'para': 'DET',
    'al': 'DET',
    'como': 'DET',
    'fue': 'DET',
    'más': 'DET',
    'lo': 'DET',
    'sus': 'DET',
    'este': 'DET',
    'ha': 'DET',
    'pero': 'DET',
    'también': 'DET',
    'esta': 'DET',
    'entre': 'DET',
    'son': 'DET',
    'dos': 'NUM',
    'le': 'DET',
    'muy': 'DET',
    'años': 'NOUN',
    'desde': 'DET',
    'año': 'NOUN',
    'hasta': 'DET',
    'era': 'DET',
    'sobre': 'DET',
    'durante': 'DET',
    'ser': 'DET',
    'eran': 'DET',
    'sin': 'DET',
    'donde': 'PROPN',
    'ya': 'DET',
    'está': 'DET',
    'tiene': 'DET',
    'parte': 'NOUN',
    'población': 'NOUN',
    'km': 'NOUN',
    'ciudad': 'NOUN'
}

# Print the resulting dictionary
print(manual_tags)


{'de': 'DET', 'la': 'DET', 'el': 'DET', 'en': 'DET', 'que': 'DET', 'los': 'DET', 'del': 'DET', 'se': 'DET', 'un': 'NUM', 'por': 'DET', 'con': 'DET', 'una': 'NUM', 'las': 'DET', 'es': 'DET', 'su': 'DET', 'para': 'DET', 'al': 'DET', 'como': 'DET', 'fue': 'DET', 'más': 'DET', 'lo': 'DET', 'sus': 'DET', 'este': 'DET', 'ha': 'DET', 'pero': 'DET', 'también': 'DET', 'esta': 'DET', 'entre': 'DET', 'son': 'DET', 'dos': 'NUM', 'le': 'DET', 'muy': 'DET', 'años': 'NOUN', 'desde': 'DET', 'año': 'NOUN', 'hasta': 'DET', 'era': 'DET', 'sobre': 'DET', 'durante': 'DET', 'ser': 'DET', 'eran': 'DET', 'sin': 'DET', 'donde': 'PROPN', 'ya': 'DET', 'está': 'DET', 'tiene': 'DET', 'parte': 'NOUN', 'población': 'NOUN', 'km': 'NOUN', 'ciudad': 'NOUN'}


In [13]:
# Manual POS tags LF
def manual_pos_tagger(doc):
    for token in doc:
        if token.text.lower() in manual_tags:
            yield token.i, token.i + 1, manual_tags[token.text.lower()]


manual_pos_lf = skweak.heuristics.FunctionAnnotator("manual_pos", manual_pos_tagger)


#### DET LF

In [14]:
# Use a lexicon of determiners
tries = skweak.gazetteers.extract_json_data("det_spanish.json")
det_lf = skweak.gazetteers.GazetteerAnnotator("determiners", tries, case_sensitive=False)


Extracting data from det_spanish.json
Populating trie for class DET (number: 142)


#### NUM LF

In [15]:
# Use a regular expression pattern to look for digits
def num_detector(doc):
    for token in doc:
        if re.search("\d+", token.text):
            yield token.i, token.i + 1, "NUM"

# Check if the token is the word of a number from 1 to 10
def num_word_detector(doc):
    for token in doc:
        if token.text.lower() in ["uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve", "diez"]:
            yield token.i, token.i + 1, "NUM"

num_lf1 = skweak.heuristics.FunctionAnnotator("numerals1", num_detector)
num_lf2 = skweak.heuristics.FunctionAnnotator("numerals2", num_word_detector)


#### PROPN LF

In [16]:
# Check if the fist letter of a word or the whole word is capitalized
def propn_detector(doc):
    for token in doc:
        if token.i == 0:
            # For the first word of a sentence, check if all letters are capitalized
            if token.text.isupper():
                yield token.i, token.i + 1, "PROPN"
        else:
            if token.text.isupper() or token.text[0].isupper():
                yield token.i, token.i + 1, "PROPN"


propn_lf = skweak.heuristics.FunctionAnnotator("proper_nouns", propn_detector)


#### ADJ LFs

In [17]:
# Look for common suffixes and prefixes
def adj_detector_suffixes(doc):
    #suffixes = ("able", "al", "ful", "ic", "ive", "less", "ous", "y", "ish", "ible", "ent", "est")
    suffixes = ("ción", "dad", "aje", "ista", "ante", "or", "ista", "ero", "ista", "oso", "ante", "ible", "al", "oso", "ino", "oso", "ar", "er", "ir", "ar", "er", "ir", "mente", "mente", "mente", "ito", "ita", "illo", "illa")

    for token in doc:
        if len(token.text) > 3 and token.text.endswith(suffixes):
            yield token.i, token.i + 1, "ADJ"


# Look for common prefixes
def adj_detector_prefixes(doc):
    #prefixes = ("un", "im", "in", "ir", "il", "non", "dis")
    prefixes = ("des", "re", "pre", "ex", "anti", "sub", "trans", "pos", "bi", "tri", "multi")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "ADJ"


# If the previous word is a form of "be" and the current word does not end with "ing" and was not labeled as DET, then it's an adjective
def adj_detector_synt1(doc):
    palabras_clave_ser = ["ser", "soy", "eres", "es", "somos", "sois", "son", "fui", "fuiste", "fue", "fuimos", "fuisteis", "fueron", "era", "eras", "éramos", "erais", "eran"]
    weak_labels = ["O"] * len(doc)
    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            prev = doc[token.i - 1].text.lower()
            #if prev in ["be", "been", "being", "am", "is", "are", "was", "were"] and (
            #        not token.text.endswith("ing")) and weak_labels[token.i] == "O":
            #    yield token.i, token.i + 1, "ADJ"
        if token.text.lower() in palabras_clave_ser and (not token.text.endswith("ando") or not token.text.endswith("iendo")) and weak_labels[token.i] == "O":
            yield token.i, token.i + 1, "ADJ"
            
# If the previous word is labeld as DET or NUM, then the current word is an adjective
def adj_detector_synt2(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals2"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "ADJ"


adj_lf1 = skweak.heuristics.FunctionAnnotator("adjectives1", adj_detector_suffixes)
adj_lf2 = skweak.heuristics.FunctionAnnotator("adjectives2", adj_detector_prefixes)
adj_lf3 = skweak.heuristics.FunctionAnnotator("adjectives3", adj_detector_synt1)
adj_lf4 = skweak.heuristics.FunctionAnnotator("adjectives4", adj_detector_synt2)


#### NOUN LF

Let's create a labeling function that looks for common noun suffixes. Can you think of some?

In [18]:
# ***********************************
def noun_detector_suffixes(doc):
    suffixes = ("ción", "dad", "dor", "dad", "ismo", "miento", "ción", "aje", "ería", "ez")
    for token in doc:
        if len(token.text) > 3 and token.text.endswith(suffixes):
            yield token.i, token.i + 1, "NOUN"
    
# ***********************************

In [19]:
# Look for common prefixes
def noun_detector_prefixes(doc):
    prefixes = (
        "anti", "auto", "bi", "co", "counter", "dis", "ex", "hyper", "in", "inter", "kilo", "mal", "mega", "mis",
        "mini", "mono", "neo", "out", "poly", "pseudo", "re", "semi", "sub", "super", "sur", "tele", "tri", "ultra",
        "under", "vice")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "NOUN"


# # If the previous word is labeld as DET, NUM or ADJ, then the current word is an noun
def noun_detector_synt(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals2"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives2"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives3"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives4"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "NOUN"


noun_lf1 = skweak.heuristics.FunctionAnnotator("nouns1", noun_detector_suffixes)
noun_lf2 = skweak.heuristics.FunctionAnnotator("nouns2", noun_detector_prefixes)
noun_lf3 = skweak.heuristics.FunctionAnnotator("nouns3", noun_detector_synt)


## Apply LFs

In [20]:
# Put all LFs in a list
lfs = [
    lexicon_lf, manual_pos_lf, det_lf, 
    num_lf1, num_lf2, propn_lf,
    adj_lf1, adj_lf2, adj_lf3, adj_lf4,
    noun_lf2, noun_lf3
]

train_docs = tag_all(train_docs, lfs)

In [21]:
# Print some of the assigned weak labels
for doc in train_docs[:3]:
    skweak.utils.display_entities(doc, ["determiners", "nouns2"])

In [22]:
# Train HMM
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm = hmm.fit(train_docs)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Finished E-step with 14187 documents
Starting iteration 2


         1    -1115622.4238             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Finished E-step with 14187 documents
Starting iteration 3


         2    -1046195.5699      +69426.8539


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Finished E-step with 14187 documents
Starting iteration 4


         3     -995387.0434      +50808.5265


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Finished E-step with 14187 documents
Starting iteration 5


         4     -957274.6232      +38112.4202


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Finished E-step with 14187 documents


         5     -937772.9840      +19501.6392


In [23]:
# Majority voting
mv = skweak.aggregation.MajorityVoter("mv", all_labels)

In [24]:
# Apply LFs, HMM and MV to the test docs
test_docs = load_data_split("test", all_labels)
test_docs = tag_all(test_docs, lfs + [mv, hmm])

## Evaluate

#### Which POS tags are easier to detect?

* We see that POS tags like determiners and numerals are easier to detect and we can achieve a good F1 score with just one or two simple LFs.


In [25]:
df = evaluate(test_docs, all_labels, [
    "determiners", "numerals1", "numerals2", "proper_nouns"
])

In [26]:
df.loc[["DET", "NUM", "PROPN"]]

tok_precision  tok_recall  tok_f1 tok_cee  \
label proportion model                                                     
DET   27.3 %     determiners           0.879       0.985   0.928           
                 numerals1             0.000       0.000   0.000           
                 numerals2             0.000       0.000   0.000           
                 proper_nouns          0.000       0.000   0.000           
NUM   4.2 %      determiners           0.000       0.000   0.000           
                 numerals1             0.951       0.769   0.850           
                 numerals2             0.647       0.096   0.168           
                 proper_nouns          0.000       0.000   0.000           
PROPN 15.2 %     determiners           0.000       0.000   0.000           
                 numerals1             0.000       0.000   0.000           
                 numerals2             0.000       0.000   0.000           
                 proper_nouns          0.700       0.901   0.788           

                              tok_acc coverage  ent_precision  ent_recall  \
label proportion model                                                      
DET   27.3 %     determiners                            0.879       0.985   
                 numerals1                              0.000       0.000   
                 numerals2                              0.000       0.000   
                 proper_nouns                           0.000       0.000   
NUM   4.2 %      determiners                            0.000       0.000   
                 numerals1                              0.951       0.769   
                 numerals2                              0.647       0.096   
                 proper_nouns                           0.000       0.000   
PROPN 15.2 %     determiners                            0.000       0.000   
                 numerals1                              0.000       0.000   
                 numerals2                              0.000       0.000   
                 proper_nouns                           0.700       0.901   

                               ent_f1  
label proportion model                 
DET   27.3 %     determiners    0.928  
                 numerals1      0.000  
                 numerals2      0.000  
                 proper_nouns   0.000  
NUM   4.2 %      determiners    0.000  
                 numerals1      0.850  
                 numerals2      0.168  
                 proper_nouns   0.000  
PROPN 15.2 %     determiners    0.000  
                 numerals1      0.000  
                 numerals2      0.000  
                 proper_nouns   0.788

* Other POS tags like adjectives and nouns, which rely more on the context are harder to detect and require more complicated rules.

In [27]:
df = evaluate(test_docs, all_labels, [
    "adjectives1", "adjectives2", "adjectives3", "adjectives4",
    "nouns1", "nouns2", "nouns3"
])

In [28]:
df.loc[["ADJ", "NOUN"]]

tok_precision  tok_recall  tok_f1 tok_cee  \
label proportion model                                                    
ADJ   12.3 %     adjectives1          0.154       0.238   0.186           
                 adjectives2          0.133       0.085   0.104           
                 adjectives3          0.000       0.000   0.000           
                 adjectives4          0.071       0.192   0.104           
                 nouns1               0.000       0.000   0.000           
                 nouns2               0.000       0.000   0.000           
                 nouns3               0.000       0.000   0.000           
NOUN  41.0 %     adjectives1          0.000       0.000   0.000           
                 adjectives2          0.000       0.000   0.000           
                 adjectives3          0.000       0.000   0.000           
                 adjectives4          0.000       0.000   0.000           
                 nouns1               0.000       0.000   0.000           
                 nouns2               0.392       0.140   0.206           
                 nouns3               0.343       0.642   0.448           

                             tok_acc coverage  ent_precision  ent_recall  \
label proportion model                                                     
ADJ   12.3 %     adjectives1                           0.154       0.238   
                 adjectives2                           0.133       0.085   
                 adjectives3                           0.000       0.000   
                 adjectives4                           0.071       0.192   
                 nouns1                                0.000       0.000   
                 nouns2                                0.000       0.000   
                 nouns3                                0.000       0.000   
NOUN  41.0 %     adjectives1                           0.000       0.000   
                 adjectives2                           0.000       0.000   
                 adjectives3                           0.000       0.000   
                 adjectives4                           0.000       0.000   
                 nouns1                                0.000       0.000   
                 nouns2                                0.392       0.140   
                 nouns3                                0.343       0.642   

                              ent_f1  
label proportion model                
ADJ   12.3 %     adjectives1   0.186  
                 adjectives2   0.104  
                 adjectives3   0.000  
                 adjectives4   0.104  
                 nouns1        0.000  
                 nouns2        0.000  
                 nouns3        0.000  
NOUN  41.0 %     adjectives1   0.000  
                 adjectives2   0.000  
                 adjectives3   0.000  
                 adjectives4   0.000  
                 nouns1        0.000  
                 nouns2        0.206  
                 nouns3        0.448

#### Which type of LF works the best?

* For adjectives the LF that uses suffixes works the best, while the syntactic rules are less accurate. On the contrary, for nouns the LF that is based on syntactic analysis has the best results. For both POS tags, the LFs that use prefixes do not yield good results.

#### Which aggregator works best?

* Despite its simplicity, majority voting outperforms HMM on almost all of the POS tags and overall achieves a higher macro F1 score.

In [29]:
df = evaluate(test_docs, all_labels, ["mv", "hmm"])

In [30]:
df

tok_precision  tok_recall  tok_f1 tok_cee tok_acc  \
label    proportion model                                                      
ADJ      12.3 %     hmm            0.073       0.188   0.106                   
                    mv             0.117       0.162   0.136                   
DET      27.3 %     hmm            0.885       0.954   0.918                   
                    mv             0.478       0.839   0.610                   
NOUN     41.0 %     hmm            0.095       0.070   0.080                   
                    mv             0.353       0.195   0.252                   
NUM      4.2 %      hmm            0.706       0.105   0.182                   
                    mv             0.920       0.603   0.728                   
PROPN    15.2 %     hmm            0.802       0.714   0.756                   
                    mv             0.810       0.597   0.688                   
macro               hmm            0.512       0.406   0.452                   
                    mv             0.536       0.479   0.506                   
micro               hmm            0.404       0.425   0.414    4.64   0.597   
                    mv             0.438       0.445   0.442   3.459   0.473   
weighted            hmm            0.441       0.425   0.432                   
                    mv             0.451       0.445   0.448                   

                          coverage  ent_precision  ent_recall  ent_f1  
label    proportion model                                              
ADJ      12.3 %     hmm                     0.073       0.188   0.106  
                    mv                      0.117       0.162   0.136  
DET      27.3 %     hmm                     0.885       0.954   0.918  
                    mv                      0.478       0.839   0.610  
NOUN     41.0 %     hmm                     0.095       0.070   0.080  
                    mv                      0.353       0.195   0.252  
NUM      4.2 %      hmm                     0.706       0.105   0.182  
                    mv                      0.920       0.603   0.728  
PROPN    15.2 %     hmm                     0.802       0.714   0.756  
                    mv                      0.810       0.597   0.688  
macro               hmm                     0.512       0.406   0.452  
                    mv                      0.536       0.479   0.506  
micro               hmm      1.053          0.404       0.425   0.414  
                    mv       1.016          0.438       0.445   0.442  
weighted            hmm                     0.441       0.425   0.432  
                    mv                      0.451       0.445   0.448

## Using Libraries as Labelling functions


In this part, we use popular NLP libraries to create labeling functions. They include Spacy, NLTK, Textblob.
We use the Majority Voter and HMM as aggregation functions
Optionally, you can train your own model on the data.

Learning goals:
- Understand how to use external libraries as labeling functions
- Understand the Spacy object and how to use it for annotation

First, read and understand the two functions below.

In [31]:

# Sometimes data formats (here POS tags) differ. We load the data and convert it to the format we need. 
# Surely, there is some loss of information
def nltk_tagger(doc):
    for token in doc:
        if not token.is_punct:
            # Tag token with nltk
            nltk_pos = nltk.pos_tag([token.text])[0][1]
            # Map nltk pos tags to ours
            if nltk_pos == "DT":
                yield token.i, token.i + 1, "DET"
            elif nltk_pos == "CD":
                yield token.i, token.i + 1, "NUM"
            elif nltk_pos == "NNP" or nltk_pos == "NNPS":
                yield token.i, token.i + 1, "PROPN"
            elif nltk_pos == "JJ" or nltk_pos == "JJR" or nltk_pos == "JJS":
                yield token.i, token.i + 1, "ADJ"
            elif nltk_pos == "NN" or nltk_pos == "NNS":
                yield token.i, token.i + 1, "NOUN"
            elif nltk_pos == "VB" or nltk_pos == "VBD" or nltk_pos == "VBG" or nltk_pos == "VBN" or nltk_pos == "VBP" or nltk_pos == "VBZ":
                yield token.i, token.i + 1, "VERB"


# We cn also use the Textblob library to get POS tags
# Under the hood, it uses the Pattern library. Once again, a transformation of the tag-labels is needed
def textblob_tagger(doc):
    for token in doc:
        if not token.is_punct:
            textblob_pos = TextBlob(token.text, pos_tagger=PatternTagger()).tags
            if len(textblob_pos) > 0:
                yield token.i, token.i + 1, penntreebank2universal(textblob_pos[0][1])


## Write the Spacy Labeling Functions

Use the two english Spacy models "en_core_web_sm", "en_core_web_md" to create labeling functions.
The challenge is that they use different tokens, i.e. the atomic units of a sentence. Our simple tokenization just splits the words by whitespace.
Your task it to design an algorithm that maps the tokens of the simple tokenization to the tokens of the Spacy tokenization, and use the token available there to create labeling functions.

Hints:
1) Access token i by `token=doc[i]` or obtain its poition by `i=token.i`
2) Access the Spacy POS token (its ground truth) by `pos=token.pos_`

In [32]:
eng_nlp_sm = spacy.load("en_core_web_sm")
eng_nlp_md = spacy.load("en_core_web_md")

# ***********************************

def eng_spacy_tagger_sm(doc):
    other_doc = eng_nlp_sm(doc.text)
    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]

def eng_spacy_tagger_md(doc):
    other_doc = eng_nlp_md(doc.text)
    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]
# ***********************************

In [32]:
eng_nlp_sm = spacy.load("en_core_web_sm")
eng_nlp_md = spacy.load("en_core_web_md")

# ***********************************

def eng_spacy_tagger_sm(doc):
    other_doc = eng_nlp_sm(doc.text)
    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]

def eng_spacy_tagger_md(doc):
    other_doc = eng_nlp_md(doc.text)
    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]
# ***********************************

In [33]:

nltk_lf = skweak.heuristics.FunctionAnnotator("nltk", nltk_tagger)
textblob_lf = skweak.heuristics.FunctionAnnotator("textblob", textblob_tagger)
eng_spacy_sm_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_sm", eng_spacy_tagger_sm)
eng_spacy_md_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_md", eng_spacy_tagger_md)

### Load Data and apply Labeling functions

Before and after applying the labeling functions, and the aggregation functions, we compute the recall and number of conflicts. For the sake of time, we use this time only a subset of the data.

In [34]:

# load training and test data
lfs = [nltk_lf, eng_spacy_sm_lf, textblob_lf, eng_spacy_md_lf]
all_labels = ["DET", "NUM", "PROPN", "NOUN", "ADJ"]

# small amount of data for the sake of time
train_docs = load_data_split("train", all_labels, 3000)

# tag the training documents
train_docs = tag_all(train_docs, lfs)



In [35]:
recall = compute_recall(train_docs)
num_conflicts = compute_num_conflicts(train_docs)
print("Train recall", recall)
print("Train conflicts", num_conflicts)

Train recall 1.0
Train conflicts 0.8035


We observe that the recall is very high. This is because the used libraries are quite well.
Further, we observe that in 40.5% of the tokens there is a conflict.

In [36]:
# train the HMM
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm=hmm.fit(train_docs)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 2


         1     -221402.1693             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 3


         2     -216998.1289       +4404.0405


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 4


         3     -215905.4287       +1092.7002


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents
Starting iteration 5


         4     -215414.7898        +490.6389


Number of processed documents: 1000
Number of processed documents: 2000
Finished E-step with 3000 documents


         5     -215178.9114        +235.8785


Now we compare how majority vote and HMM change the number of conflicts.
Remember, that it's important to set Majority vote before HMM, otherwise Majority Vote takes the HMM predictions into account

In [37]:
mv = skweak.aggregation.MajorityVoter("mv", all_labels)
train_docs = tag_all(train_docs, [mv, hmm])

num_conflicts = compute_num_conflicts(train_docs)
print("Conflicts with MV on train set: ", num_conflicts)

Conflicts with MV on train set:  0.8035


We observe that the number of token conflicts does not change. The reason is that both methods can not choose a class different from the labeling functions.

## Evaluation

Look at the Precision, Recall and F1-Score of the different aggregation functions. What do you observe?

In [38]:
# tag the test documents
# Once again, it's important to set Majority vote before HMM, otherwise Majority Vote takes the HMM predictions into account
test_docs = load_data_split("test", all_labels, 1000)
test_docs = tag_all(test_docs, lfs + [mv, hmm])

num_conflicts = compute_num_conflicts(test_docs)
print("Conflicts on test set", num_conflicts)

Conflicts on test set 0.8114


In [39]:
df = evaluate(test_docs, all_labels, [ "mv", "hmm"])

In [40]:
df

tok_precision  tok_recall  tok_f1 tok_cee tok_acc  \
label    proportion model                                                      
ADJ      12.3 %     hmm            0.600       0.134   0.220                   
                    mv             0.656       0.088   0.156                   
DET      27.3 %     hmm            0.017       0.003   0.006                   
                    mv             0.020       0.004   0.006                   
NOUN     41.0 %     hmm            0.251       0.412   0.312                   
                    mv             0.285       0.345   0.312                   
NUM      4.2 %      hmm            1.000       0.734   0.846                   
                    mv             0.988       0.734   0.842                   
PROPN    15.2 %     hmm            0.136       0.932   0.238                   
                    mv             0.122       0.955   0.216                   
macro               hmm            0.401       0.443   0.420                   
                    mv             0.414       0.425   0.420                   
micro               hmm            0.196       0.359   0.254   6.394   0.312   
                    mv             0.184       0.329   0.236   6.273   0.267   
weighted            hmm            0.244       0.358   0.290                   
                    mv             0.263       0.329   0.292                   

                          coverage  ent_precision  ent_recall  ent_f1  
label    proportion model                                              
ADJ      12.3 %     hmm                     0.600       0.134   0.220  
                    mv                      0.656       0.088   0.156  
DET      27.3 %     hmm                     0.017       0.003   0.006  
                    mv                      0.020       0.004   0.006  
NOUN     41.0 %     hmm                     0.251       0.412   0.312  
                    mv                      0.285       0.345   0.312  
NUM      4.2 %      hmm                     1.000       0.734   0.846  
                    mv                      0.988       0.734   0.842  
PROPN    15.2 %     hmm                     0.136       0.932   0.238  
                    mv                      0.122       0.955   0.216  
macro               hmm                     0.401       0.443   0.420  
                    mv                      0.414       0.425   0.420  
micro               hmm      1.829          0.196       0.359   0.254  
                    mv        1.79          0.184       0.329   0.236  
weighted            hmm                     0.244       0.358   0.290  
                    mv                      0.263       0.329   0.292

Contrary, to the first part, we observe that the HMM performs better than majority vote.